In [146]:
import os
import pywt
import math
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from scipy.stats import norm

In [230]:
# files = os.listdir("spectra")

# for filename in files:
#     np.genfromtxt(filename, delimiter=",")

limit_low = None
limit_high = None

spectrum = np.genfromtxt("spectra/E (2).TXT", delimiter=",")
spectrum = np.transpose(spectrum)
wavenumbers = spectrum[0]
intensities = spectrum[1]

if limit_low is not None:
    limit_low_index = list(wavenumbers).index(limit_low)
else:
    limit_low_index = 0
    limit_low = wavenumbers[0]

if limit_high is not None:
    limit_high_index = list(wavenumbers).index(limit_high)
else:
    limit_high_index = len(wavenumbers)
    limit_high = wavenumbers[-1]

wavenumbers = wavenumbers[limit_low_index:limit_high_index]
intensities = intensities[limit_low_index:limit_high_index]

In [231]:
intensities_sg = sig.savgol_filter(intensities, 
                                   window_length=3, 
                                   polyorder=0, 
                                   deriv=0)

In [232]:
%matplotlib
fig, ax = plt.subplots()
ax.plot(wavenumbers, intensities_sg, linewidth = 1, color = "red")
ax.set_xlim(limit_low, limit_high)
ax.invert_xaxis()
ax.grid()

Using matplotlib backend: Qt5Agg


In [233]:
intensities_ext = np.concatenate((np.full(400, intensities_sg[0]), intensities_sg, np.full(400, intensities_sg[-1])))
scales = np.linspace(1,200, 200)

coefs, freqs = pywt.cwt(data=intensities_ext, scales=scales, wavelet="mexh")

coefs = coefs[:,400:-400]

In [234]:
wavenumbers_grid, scales_grid = np.meshgrid(wavenumbers, scales)

In [235]:
%matplotlib
fig, [ax1, ax2] = plt.subplots(2,1)
sp = ax1.plot(wavenumbers, intensities_sg, linewidth=1, color="red")
ax1.set_xlim(limit_low, limit_high)
ax1.invert_xaxis()
ax1.grid()

cmap = cm.seismic
normal = colors.Normalize(vmin=-np.max(np.abs(coefs)), vmax=np.max(np.abs(coefs)))

cp = ax2.contourf(wavenumbers_grid, scales_grid, coefs, cmap=cmap, norm=normal, levels=200, extend="both")
cbar = fig.colorbar(cp, orientation="horizontal")
ax2.invert_xaxis()

Using matplotlib backend: Qt5Agg


In [236]:
coefs_sum_by_scale = abs(coefs).sum(axis=1, keepdims=True)
p_coefs = abs(coefs) / coefs_sum_by_scale

shannon_H = -np.sum(p_coefs * np.log(p_coefs), axis=1)

In [237]:
plt.plot(scales, shannon_H)

In [238]:
mins = np.array(sig.argrelextrema(shannon_H, np.less))
scale_choice = scales[mins[0,0]]

In [239]:
intensities_ext = np.concatenate((np.full(400, intensities_sg[0]), intensities_sg, np.full(400, intensities_sg[-1])))
coefs, freqs = pywt.cwt(data=intensities_ext, scales=scale_choice, wavelet="mexh")
coefs = coefs[0,400:-400]

In [245]:
%matplotlib
fig, ax= plt.subplots()

ax.plot(wavenumbers, intensities_sg, linewidth = 0.5, color = "red")
ax.set_xlim(limit_low, limit_high)
ax.invert_xaxis()

ax.plot(wavenumbers, coefs, linewidth = 0.5, color = "blue")
ax.set_xlim(limit_low, limit_high)
ax.invert_xaxis()
ax.grid()

Using matplotlib backend: Qt5Agg


In [246]:
def iterative_gaussian_fit(data, n_bins=200, diff=0.05):
    data_range = np.max(data) - np.min(data)
    n_change_prev = 0
    
    while True:
        # Calculate histogram
        hist, bins = np.histogram(data, n_bins)
        
        # Find bins with density more than 1/3 of max density
        max_bins_indices = np.where(hist > max(hist)/3)
        max_bins_low = max_bins_indices[0][0]
        max_bins_high = max_bins_indices[0][-1]
        max_data_low = bins[max_bins_low]
        max_data_high = bins[max_bins_high+1]
        
        # Reduce data to selected bins and calculate mean and standard deviation
        data_reduced = data[data > max_data_low]
        data_reduced = data_reduced[data_reduced < max_data_high]
        mu, sigma = norm.fit(data_reduced, floc=0)
        
        #Calculate new number of bins nad compare difference
        n_bins_prev = n_bins
        n_bins = math.ceil(8 * data_range / sigma)
        n_change = (n_bins/n_bins_prev)-1
        print("Change:", round(n_change*100, 1), "%")
        if abs(n_change) <= diff or n_change*n_change_prev < 0:
            return sigma
        n_change_prev = n_change
        

In [248]:
iterative_gaussian_fit(coefs)

Change: 6097.5 %
Change: 541.4 %
Change: 8.4 %
Change: -2.4 %


44.71061574125781